In [1]:
import pathlib as pl
import pandas as pd
import re

pd.set_option('future.no_silent_downcasting', True)

%cd -q "/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/notebooks"
_PROJECT_CONFIG_NB = str(pl.Path("00_project_config.ipynb").resolve(strict=True))

%run $_PROJECT_CONFIG_NB

_MYNAME="dump-busco-table"
_NBSTAMP=get_nb_stamp(_MYNAME)

ASSEMBLER = "verkko"

# from NB:
# plot-busco-stores
cache_file = PROJECT_NB_CACHE.joinpath(f"compleasm-{ASSEMBLER}.cache.tsv.gz")

table = pd.read_csv(cache_file, sep="\t", header=0, skiprows=1)

table["sample"] = table["sample"].apply(lambda x: x.split(".")[0])
table.drop("delta_n", axis=1, inplace=True)

table["karyotype"] = table["karyotype"].replace({"unk": "(unphased)"}, inplace=False)

table = table[
    [
        "sample", "asm_unit", "karyotype",
        "total", "adj_total",
        "single", "single_pct",
        "duplicated", "duplicated_pct",
        "fragmented", "fragmented_pct",
        "missing", "missing_pct"
        
    ]
]

renamer = {
    "sample": "sample",
    "asm_unit": "assembly unit",
    "karyotype": "assembly sex",
    "single": "single-copy genes (n)",
    "duplicated": "duplicated genes (n)",
    "missing": "missing genes (n)",
    "fragmented": "fragmented genes (n)",
    "adj_total": "adj. total genes (n)",
    "single_pct": "single-copy genes (%)",
    "duplicated_pct": "duplicated genes (%)",
    "fragmented_pct": "fragmented genes (%)",
    "missing_pct": "missing genes (%)",
    "total": "total genes (n)"
}

table.rename(renamer, axis=1, inplace=True)

table_sx_busco_scores = TABLE_OUT_SUPPL.joinpath(
    f"table_SXPE_{ASSEMBLER}-busco-scores.tsv"
)

table.to_csv(table_sx_busco_scores, sep="\t", header=True, index=False)